<a href="https://colab.research.google.com/github/sda96/AIFFEL_3rd_hackerton_TUNiB_DKTC/blob/main/notebook/SeungTaek/json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!gdown https://drive.google.com/uc?id=15NHQXINo1xILHiBvnCWf1XqCtrxlevtx
!gdown https://drive.google.com/uc?id=1dvjlJlEl3VQ4mhd9CybqeEs2Uw1wzqW_

Downloading...
From: https://drive.google.com/uc?id=15NHQXINo1xILHiBvnCWf1XqCtrxlevtx
To: /content/SNS_train.zip
100% 650M/650M [00:04<00:00, 130MB/s]
Downloading...
From: https://drive.google.com/uc?id=1dvjlJlEl3VQ4mhd9CybqeEs2Uw1wzqW_
To: /content/SNS_valid.zip
100% 81.3M/81.3M [00:01<00:00, 80.1MB/s]


In [5]:
!unzip -uq "/content/SNS_train.zip"
!unzip -uq "/content/SNS_valid.zip"

In [6]:
import pandas as pd # pandas 모듈 로드
import json  # json 모듈 로드
import re

In [7]:
json_file_path="/content/[라벨]한국어SNS_train/개인및관계.json"

In [8]:
def preprocess_sentence(sentence):

    sentence = re.sub(r"[^0-9]+", " ", sentence)

    return sentence

In [9]:
def yield_json(path):

    with open(path, 'r', encoding='utf-8-sig') as j: #파일을 열어서 읽기 : r 

        data = json.loads(j.read()) # json파일을 dictionary 형태로 읽어서 data에 넣어준다.

        for i in range(len(data['data'])):
            df = pd.json_normalize(data['data'][i]["body"])
            df['index'] = 0
            df['turnID'] = df['turnID'].apply(preprocess_sentence)
            df['turnID'].astype(int)
            df = df.groupby(['turnID', 'utterance']).sum().groupby(level=[0]).cumsum()
            yield df

In [10]:
df = yield_json(json_file_path)

In [11]:
next(df)

index
turnID utterance                             
 1     그런생각이들어                              0
       근데지금은                                0
       나는 이해하면 안돼는거까지 이해햇엇구나                0
       내가 전남친이랑헤어지면서 깨달앗거든                  0
       내가또 똑같은 실수를 하는건 아닌가                  0
       라는 생각이들어                             0
       오빨 너무 좋아해서 이해하면 안돼는거까지 이해하고있는건가      0
       요새 자꾸                                0
 2     그럼 내가                                0
       할말이 없네                               0
 3     고치겟다던가                               0
       그런생각들게해서 미안하다던가                      0
       대답도 그렇게안햇으면좋겟어                       0
       뭐때문에그렇게생각하냐던가                        0
       여기서 할말없다고 하면 나는 고민하고 한말인데            0
       이렇게말해야지                              0
       할말없다 이런말 싫어                          0
 4     무슨말을하겠어                              0
       어떻게                                  0
       전남친이랑 똑같은 실수인가싶다는데                   0
 5     나 진지하게 얘기하고싶어서 꺼낸말이야                 0
       실망하고                                 0
       요새자꾸 그런생각이들어서..                      0
       이해하고넘어간부분은                           0
       전혀다르긴한데                              0
 6     그런생각들게 해서                            0
       미안해                                  0
 7     뭐때문인지는 안궁금해??                        0
 8     뭐때문이야??                              0
 9     가끔 짜증너무많이낸다거나                        0

In [ ]:
len(contents['data'])

In [ ]:
for i in range(len(contents['data'][1]['body'])):
    print(contents['data'][1]['body'][i]['participantID'])  
    print(contents['data'][1]['body'][i]['utterance'])